In [1]:
import pandas as pd
import numpy as np
import copy

# Download Employment Center Data

In [2]:
ec_list = pd.read_csv('ec_list.csv')

# Trip Summary Data Excel

In [3]:
replica = pd.read_csv("replica-first_study-03_21_23-trips_dataset.csv")#, nrows=10000)
replica['dummy_val'] = 1

In [4]:
replica['trip_duration_minutes'] = replica['trip_duration_minutes'].replace(0,1)
replica['trip_duration'] = pd.cut(x=replica['trip_duration_minutes'], bins=[0,5,15,30,60,np.inf], labels=['<=5 mins', '>5-15 mins', '>15-30 mins','>30-60 mins','>60 mins'], ordered =True)

In [5]:
replica['trip_distance'] = pd.cut(x=replica['trip_distance_miles'], bins=[-1,1,2,5,10,25,np.inf], labels=['<=1 mile','>1-2 miles','>2-5 miles','>5-10 miles','>10-25 miles','>25 miles'], ordered =True)

In [6]:
input_dictionaries = {
    'trips_by_primary_mode': {'agg_column':'primary_mode', 'column_header': 'Mode'},
    'trips_by_purpose': {'agg_column': 'trip_purpose', 'column_header': 'Trip Purpose'},
    'trips_by_previous_trip': {'agg_column':'previous_trip_purpose', 'column_header': 'Prev Trip Purpose'},
    'trips_by_vehicle_type': {'agg_column': 'vehicle_type', 'column_header': 'Vehicle Type'},
    'trips_by_origin_land_use': {'agg_column': 'origin_land_use', 'column_header': 'Origin Land Use'}, 
    'trips_by_destination_land_use': {'agg_column': 'destination_land_use', 'column_header': 'Destination Land Use'},
    'trips_by_destination_builidng_use': {'agg_column': 'destination_building_use', 'column_header': 'Destination Building Use'},
    'trips_by_duration': {'agg_column': 'trip_duration', 'column_header': 'Trip Duration'},
    'trips_by_distance': {'agg_column': 'trip_distance', 'column_header': 'Trip Distance'},
    'trips_by_origin_building_use': {'agg_column': 'origin_building_use', 'column_header': 'Origin Building Use'}
}

In [7]:
def number_of_trips_by_manipulation(agregation_column):
    groupby_vals = ['destination_custom', agregation_column] 
    temp_df = replica[groupby_vals + ['dummy_val']]
    grouped = temp_df.groupby(groupby_vals).agg({'count'})
    grouped = grouped.reset_index()
    grouped.columns = ['employment_center', 'agg_column', 'count']
    return grouped

In [8]:
def pivot_table_output(df):
    output = pd.pivot_table(df, values='count', index=['employment_center'], columns=['agg_column'])
    output.columns.name = ''
    return output

In [9]:
def add_header(df, column_header):
    header = pd.MultiIndex.from_product([[column_header], df.columns])
    df.columns = header
    return df

In [10]:
def create_output(key):
    grouped_data = number_of_trips_by_manipulation(agregation_column=input_dictionaries[key]['agg_column'])
    
    pivoted_table = pivot_table_output(grouped_data)
    if key == 'trips_by_previous_trip':
        pivoted_table = pivoted_table.rename(columns={'\\N': 'unknown'})
        #pivoted_table = pivoted_table.drop('\\N', axis = 1)

    header_added = add_header(df=pivoted_table, column_header=input_dictionaries[key]['column_header'])

    return header_added

In [11]:
output_df = pd.DataFrame()

for input_key in input_dictionaries.keys():
    if output_df.empty:
        output_df = create_output(key=input_key)
        output_df['Total'] = output_df.sum(axis=1)
    else:
        output_df = output_df.merge(create_output(key=input_key), how='inner', left_index=True, right_index=True)
    print(f"{input_key}: {create_output(key=input_key).shape}")

# Merge output with the Employment Centers data 
output_df = output_df.merge(ec_list, how='left', left_index=True, right_on='EC_Name')

# Sort by EC ID values
output_df['EC_ID'] = output_df['EC_ID'].astype(int)
output_df = output_df.sort_values(['EC_ID']).reset_index(drop=True)
output_df = output_df.set_index(['EC_ID', 'EC_Name', 'Tier'])

# Turn back into a multiindexed columns
output_df.columns = pd.MultiIndex.from_tuples(output_df.columns)

# Move the "Total" column to the front of the dataframe
col = output_df.pop(('Total',''))
output_df.insert(0, ('Total',''), col)

trips_by_primary_mode: (102, 8)
trips_by_purpose: (102, 13)
trips_by_previous_trip: (102, 13)
trips_by_vehicle_type: (102, 4)
trips_by_origin_land_use: (102, 15)
trips_by_destination_land_use: (102, 15)
trips_by_destination_builidng_use: (102, 14)
trips_by_duration: (102, 5)
trips_by_distance: (102, 6)
trips_by_origin_building_use: (102, 14)


c:\Users\cra\Anaconda3\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)
  return merge(


Mode                      \
                                      biking  carpool commercial   
EC_ID EC_Name                    Tier                              
1     San Diego Airport          3     426.0  34410.0     4567.0   
2     Alpine                     4      75.0   4283.0     1510.0   
3     Barrio Logan               4     153.0   5949.0     1846.0   
4     Carlsbad Palomar Airport   2     502.0  30703.0     4802.0   
5     Carlsbad State Beach       3     137.0  21568.0     1749.0   
...                                      ...      ...        ...   
104   Jamul Casino               6       4.0    772.0      293.0   
105   Pala Casino Spa Resort     6      24.0   2713.0      130.0   
106   Sycuan Casino Resort       6       5.0   1586.0      123.0   
107   Valley View Casino & Hotel 6       2.0   1160.0      442.0   
108   Viejas Casino & Resort     6       7.0   1315.0      272.0   

                                                                        \
                                      on_demand_auto other_travel_mode   
EC_ID EC_Name                    Tier                                    
1     San Diego Airport          3            4583.0           25987.0   
2     Alpine                     4             316.0             288.0   
3     Barrio Logan               4            1129.0             609.0   
4     Carlsbad Palomar Airport   2             524.0            1131.0   
5     Carlsbad State Beach       3             659.0            1562.0   
...                                              ...               ...   
104   Jamul Casino               6              78.0              95.0   
105   Pala Casino Spa Resort     6              55.0             429.0   
106   Sycuan Casino Resort       6             325.0             269.0   
107   Valley View Casino & Hotel 6              71.0             134.0   
108   Viejas Casino & Resort     6             293.0             191.0   

                                                                            \
                                      private_auto public_transit  walking   
EC_ID EC_Name                    Tier                                        
1     San Diego Airport          3         86488.0         2606.0  16581.0   
2     Alpine                     4         14130.0           62.0   2427.0   
3     Barrio Logan               4         25423.0         3580.0   5840.0   
4     Carlsbad Palomar Airport   2         46240.0          350.0   8234.0   
5     Carlsbad State Beach       3         28484.0          335.0   4873.0   
...                                            ...            ...      ...   
104   Jamul Casino               6          3449.0           29.0    324.0   
105   Pala Casino Spa Resort     6          2837.0           18.0    770.0   
106   Sycuan Casino Resort       6          4269.0            NaN    818.0   
107   Valley View Casino & Hotel 6          1798.0           14.0    209.0   
108   Viejas Casino & Resort     6          4119.0           21.0    652.0   

                                          Total Trip Purpose  ...  \
                                                  commercial  ...   
EC_ID EC_Name                    Tier                         ...   
1     San Diego Airport          3     175648.0       4661.0  ...   
2     Alpine                     4      23091.0       1523.0  ...   
3     Barrio Logan               4      44529.0       1909.0  ...   
4     Carlsbad Palomar Airport   2      92486.0       4854.0  ...   
5     Carlsbad State Beach       3      59367.0       1858.0  ...   
...                                         ...          ...  ...   
104   Jamul Casino               6       5044.0        300.0  ...   
105   Pala Casino Spa Resort     6       6976.0        132.0  ...   
106   Sycuan Casino Resort       6       7395.0        123.0  ...   
107   Valley View Casino & Hotel 6       3830.0        485.0  ...   
108   Viejas Casino & Resort     6       6870.0        276.0

In [23]:
# Move the "Total" column to the front of the dataframe
col = output_df.pop(('Total',''))
output_df.insert(0, ('Total',''), col)

Total   Mode                      \
                                                biking  carpool commercial   
EC_ID EC_Name                    Tier                                        
1     San Diego Airport          3     175648.0  426.0  34410.0     4567.0   
2     Alpine                     4      23091.0   75.0   4283.0     1510.0   
3     Barrio Logan               4      44529.0  153.0   5949.0     1846.0   
4     Carlsbad Palomar Airport   2      92486.0  502.0  30703.0     4802.0   
5     Carlsbad State Beach       3      59367.0  137.0  21568.0     1749.0   
...                                         ...    ...      ...        ...   
104   Jamul Casino               6       5044.0    4.0    772.0      293.0   
105   Pala Casino Spa Resort     6       6976.0   24.0   2713.0      130.0   
106   Sycuan Casino Resort       6       7395.0    5.0   1586.0      123.0   
107   Valley View Casino & Hotel 6       3830.0    2.0   1160.0      442.0   
108   Viejas Casino & Resort     6       6870.0    7.0   1315.0      272.0   

                                                                        \
                                      on_demand_auto other_travel_mode   
EC_ID EC_Name                    Tier                                    
1     San Diego Airport          3            4583.0           25987.0   
2     Alpine                     4             316.0             288.0   
3     Barrio Logan               4            1129.0             609.0   
4     Carlsbad Palomar Airport   2             524.0            1131.0   
5     Carlsbad State Beach       3             659.0            1562.0   
...                                              ...               ...   
104   Jamul Casino               6              78.0              95.0   
105   Pala Casino Spa Resort     6              55.0             429.0   
106   Sycuan Casino Resort       6             325.0             269.0   
107   Valley View Casino & Hotel 6              71.0             134.0   
108   Viejas Casino & Resort     6             293.0             191.0   

                                                                            \
                                      private_auto public_transit  walking   
EC_ID EC_Name                    Tier                                        
1     San Diego Airport          3         86488.0         2606.0  16581.0   
2     Alpine                     4         14130.0           62.0   2427.0   
3     Barrio Logan               4         25423.0         3580.0   5840.0   
4     Carlsbad Palomar Airport   2         46240.0          350.0   8234.0   
5     Carlsbad State Beach       3         28484.0          335.0   4873.0   
...                                            ...            ...      ...   
104   Jamul Casino               6          3449.0           29.0    324.0   
105   Pala Casino Spa Resort     6          2837.0           18.0    770.0   
106   Sycuan Casino Resort       6          4269.0            NaN    818.0   
107   Valley View Casino & Hotel 6          1798.0           14.0    209.0   
108   Viejas Casino & Resort     6          4119.0           21.0    652.0   

                                      Trip Purpose  ... Origin Building Use  \
                                        commercial  ...          industrial   
EC_ID EC_Name                    Tier               ...                       
1     San Diego Airport          3          4661.0  ...              1957.0   
2     Alpine                     4          1523.0  ...               322.0   
3     Barrio Logan               4          1909.0  ...              1455.0   
4     Carlsbad Palomar Airport   2          4854.0  ...              4264.0   
5     Carlsbad State Beach       3          1858.0  ...              1281.0   
...                                            ...  ...                 ...   
104   Jamul Casino               6           300.0  ...                53.0   
105   Pala Casino Spa Resort 

In [14]:
output_df.to_csv('trip_summary_data_integers.csv')

In [ ]:
output_df.head()

# Percent Output

In [26]:
percent_df = copy.deepcopy(output_df)

# define the column to divide by
col_to_divide_by = ('Total','')

# divide all columns by the column defined above
df_result = percent_df.div(percent_df[col_to_divide_by], axis=0)*100
df_result = df_result.round(1)

# update the original DataFrame by replacing all columns except the original one with the divided values
percent_df[percent_df.columns.difference([col_to_divide_by])] = df_result[df_result.columns.difference([col_to_divide_by])]

Total   Mode                     \
                                                biking carpool commercial   
EC_ID EC_Name                    Tier                                       
1     San Diego Airport          3     175648.0    0.2    19.6        2.6   
2     Alpine                     4      23091.0    0.3    18.5        6.5   
3     Barrio Logan               4      44529.0    0.3    13.4        4.1   
4     Carlsbad Palomar Airport   2      92486.0    0.5    33.2        5.2   
5     Carlsbad State Beach       3      59367.0    0.2    36.3        2.9   
...                                         ...    ...     ...        ...   
104   Jamul Casino               6       5044.0    0.1    15.3        5.8   
105   Pala Casino Spa Resort     6       6976.0    0.3    38.9        1.9   
106   Sycuan Casino Resort       6       7395.0    0.1    21.4        1.7   
107   Valley View Casino & Hotel 6       3830.0    0.1    30.3       11.5   
108   Viejas Casino & Resort     6       6870.0    0.1    19.1        4.0   

                                                                        \
                                      on_demand_auto other_travel_mode   
EC_ID EC_Name                    Tier                                    
1     San Diego Airport          3               2.6              14.8   
2     Alpine                     4               1.4               1.2   
3     Barrio Logan               4               2.5               1.4   
4     Carlsbad Palomar Airport   2               0.6               1.2   
5     Carlsbad State Beach       3               1.1               2.6   
...                                              ...               ...   
104   Jamul Casino               6               1.5               1.9   
105   Pala Casino Spa Resort     6               0.8               6.1   
106   Sycuan Casino Resort       6               4.4               3.6   
107   Valley View Casino & Hotel 6               1.9               3.5   
108   Viejas Casino & Resort     6               4.3               2.8   

                                                                           \
                                      private_auto public_transit walking   
EC_ID EC_Name                    Tier                                       
1     San Diego Airport          3            49.2            1.5     9.4   
2     Alpine                     4            61.2            0.3    10.5   
3     Barrio Logan               4            57.1            8.0    13.1   
4     Carlsbad Palomar Airport   2            50.0            0.4     8.9   
5     Carlsbad State Beach       3            48.0            0.6     8.2   
...                                            ...            ...     ...   
104   Jamul Casino               6            68.4            0.6     6.4   
105   Pala Casino Spa Resort     6            40.7            0.3    11.0   
106   Sycuan Casino Resort       6            57.7            NaN    11.1   
107   Valley View Casino & Hotel 6            46.9            0.4     5.5   
108   Viejas Casino & Resort     6            60.0            0.3     9.5   

                                      Trip Purpose  ... Origin Building Use  \
                                        commercial  ...          industrial   
EC_ID EC_Name                    Tier               ...                       
1     San Diego Airport          3             2.7  ...                 1.1   
2     Alpine                     4             6.6  ...                 1.4   
3     Barrio Logan               4             4.3  ...                 3.3   
4     Carlsbad Palomar Airport   2             5.2  ...                 4.6   
5     Carlsbad State Beach       3             3.1  ...                 2.2   
...                                            ...  ...                 ...   
104   Jamul Casino               6             5.9  ...                 1.1   
105   Pala Casino Spa Resort     6             1.9  ...  

In [ ]:
percent_df.to_csv('trip_summary_data_percents.csv')

In [ ]:
percent_df.head()

# Employment Centers Trips by Block Group CSV

In [12]:
replica_2 = pd.read_csv("replica-num_trips_blockgrps_to_ecs_fall_19_thusrday-03_21_23-trips_dataset.csv", nrows=10000)

In [13]:
trips_bgrp_to_EC = replica_2.groupby(['origin_bgrp','destination_custom']).agg({'count'}).reset_index()[['origin_bgrp','destination_custom','origin_cty']].rename(columns={'origin_cty':'trips'})

In [14]:
subset = trips_bgrp_to_EC[trips_bgrp_to_EC['origin_bgrp'].str.contains('San Diego')]
subset.columns = ['origin_bgrp', 'destination_custom', 'trip_count']
output = subset.merge(ec_list, how='left', left_on='destination_custom', right_on='EC_Name')
output['Tract'] = output['origin_bgrp'].str.extract(r'Tract\s(\d+(?:\.\d+)?)')[0]
output['BG'] = output['origin_bgrp'].str.extract(r'(\d+)')
output = output[['EC_ID', 'destination_custom', 'Tract', 'BG', 'trip_count']]
output['Tract'] = output['Tract'].astype(float)
output['BG'] = output['BG'].astype(int)
output['EC_ID'] = output['EC_ID'].astype(int)
output = output.sort_values(['EC_ID', 'Tract']).reset_index(drop=True)
output.columns = ['EC ID', 'Emp Ctr', 'Tract', 'BG', 'Trips']

,EC ID,Emp Ctr,Tract,BG,Trips
0,1,San Diego Airport,191.07,1,3
1,1,San Diego Airport,208.01,2,1
2,3,Barrio Logan,191.07,1,1
3,4,Carlsbad Palomar Airport,171.09,2,1
4,4,Carlsbad Palomar Airport,178.11,2,1
...,...,...,...,...,...
1019,107,Valley View Casino & Hotel,191.06,3,18
1020,107,Valley View Casino & Hotel,191.07,1,20
1021,107,Valley View Casino & Hotel,201.03,1,3
1022,107,Valley View Casino & Hotel,207.09,1,1


In [ ]:
output.to_csv('EC_Trips_by_BG.csv')

In [ ]:
output.head()